In [37]:
import pandas as pd
import tqdm
import numpy as np
from EduData import get_data
from sklearn.model_selection import KFold   # 不弄k-fold交叉验证就不用这个
import pickle

In [38]:
# 定义数据存放base路径
basePath='../../data/assist09/'

In [39]:
# 如果还没下载数据集原始文件，可以从edudata在线下载
get_data('assistment-2009-2010-skill',basePath)

downloader, INFO http://base.ustc.edu.cn/data/ASSISTment/2009_skill_builder_data_corrected.zip is saved as ../../data/assist09/2009_skill_builder_data_corrected.zip
downloader, INFO file existed, skipped


'../../data/assist09/2009_skill_builder_data_corrected'

In [40]:
data = pd.read_csv(basePath+'2009_skill_builder_data_corrected/skill_builder_data_corrected.csv')

/home/hejunliang/.conda/envs/hjlTorch1/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [41]:
print(data.columns)

Index(['order_id', 'assignment_id', 'user_id', 'assistment_id', 'problem_id',
       'original', 'correct', 'attempt_count', 'ms_first_response',
       'tutor_mode', 'answer_type', 'sequence_id', 'student_class_id',
       'position', 'type', 'base_sequence_id', 'skill_id', 'skill_name',
       'teacher_id', 'school_id', 'hint_count', 'hint_total', 'overlap_time',
       'template_id', 'answer_id', 'answer_text', 'first_action',
       'bottom_hint', 'opportunity', 'opportunity_original'],
      dtype='object')


In [42]:
# 自己决定要用哪些列
data = pd.read_csv(
    basePath+'2009_skill_builder_data_corrected/skill_builder_data_corrected.csv',
    usecols=['order_id', 'user_id', 'problem_id', 'skill_id','template_id', 'correct']
).dropna(subset=['order_id', 'user_id', 'problem_id', 'skill_id', 'template_id','correct'])

In [43]:
# 建立练习映射,编号从1~n
raw_problem=data.problem_id.unique().tolist()
raw_problem.sort()
num_problem=len(raw_problem)
problems={p:i+1 for i,p in enumerate(raw_problem)}
print("number of problems: %d" % num_problem)
np.save(basePath+'map/eMap.npy',problems)

number of problems: 17751


In [44]:
#将problem_id替换成eMap的value
data['problem_id']=data['problem_id'].map(problems)

In [45]:
# 建立技能映射，编号从0~n-1
raw_question = data.skill_id.unique().tolist()
num_skill = len(raw_question)
skills = { p: i for i, p in enumerate(raw_question) }
print("number of skills: %d" % num_skill)
np.save(basePath+'map/cMap.npy',skills)

number of skills: 123


In [46]:
#将skill_id替换成cMap的value
data['skill_id']=data['skill_id'].map(skills)

In [47]:
# 建立练习-技能-邻接矩阵
adj_problem_skill=np.zeros((num_problem+1,num_skill))
single_problem_skill_pair=data.drop_duplicates(subset=['problem_id','skill_id'])[['problem_id','skill_id']].sort_values(by=['problem_id'])
print(single_problem_skill_pair)
for i,row in single_problem_skill_pair.iterrows():
    adj_problem_skill[int(row['problem_id'])][int(row['skill_id'])]=1
#保存e2c邻接矩阵
np.save(basePath+'adj/e2cAdj.npy',adj_problem_skill)

        problem_id  skill_id
146242           1        39
191473           2        49
112391           3        31
191472           4        49
112390           4        31
...            ...       ...
253931       17747        81
253932       17748        81
253942       17749        81
253933       17750        81
72043        17751        20

[21246 rows x 2 columns]


In [48]:
# 建立template_id映射，编号从0~n-1
raw_template = data.template_id.unique().tolist()
num_template = len(raw_template)

templates = { p: i for i, p in enumerate(raw_template) }
print("number of templates: %d" % num_template)
np.save(basePath+'map/tMap.npy',templates)

number of templates: 658


In [49]:
data['template_id']=data['template_id'].map(templates)

In [50]:
# 建立练习-template-邻接矩阵
adj_problem_template=np.zeros((num_problem+1,num_template))
single_problem_template_pair=data.drop_duplicates(subset=['problem_id','template_id'])[['problem_id','template_id']].sort_values(by=['problem_id'])
print(single_problem_template_pair)
for i,row in single_problem_template_pair.iterrows():
    adj_problem_template[int(row['problem_id'])][int(row['template_id'])]=1
#保存e2c邻接矩阵
np.save(basePath+'adj/e2tAdj.npy',adj_problem_template)

        problem_id  template_id
146242           1          238
191473           2          238
112391           3          238
112390           4          238
146278           5          239
...            ...          ...
253931       17747          464
253932       17748          467
253942       17749          467
253933       17750          467
72043        17751          179

[17751 rows x 2 columns]


In [51]:
# 将每个学生的答题序列分好块
def parse_all_seq(students):
    all_sequences = []
    for student_id in tqdm.tqdm(students, 'parse student sequence:\t'):
        student_sequence = parse_student_seq(data[data.user_id == student_id])
        all_sequences.extend([student_sequence])
    return all_sequences


def parse_student_seq(student):
    seq = student.sort_values('order_id')
    return seq['problem_id'].values,seq['correct'].values

data=data.drop_duplicates(subset=['order_id'])  # 根据要求看要不要去掉重复记录
sequences = parse_all_seq(sorted(data.user_id.unique()))
print('总的学生人数：',len(sequences))

parse student sequence:	:   0%|          | 0/4163 [00:00<?, ?it/s]

parse student sequence:	: 100%|██████████| 4163/4163 [00:02<00:00, 1396.23it/s]

总的学生人数： 4163


In [52]:
print(type(sequences[0]))
print(type(sequences))
print(sequences[0])

<class 'tuple'>
<class 'list'>
(array([12668, 12692, 12685, 12704, 12705, 12700, 12708,  2993,  3182,
        2977,  3173,  3168, 12032, 12242, 12231, 11732, 12213, 11712,
       11715]), array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]))


In [53]:
# k-fold交叉校验
kf = KFold(n_splits=5,shuffle=True)
allFoldSeq=[]

for train_index,test_index in kf.split(sequences):
    allFoldSeq.append((np.array(sequences)[train_index], np.array(sequences)[test_index]))

/tmp/ipykernel_1236135/1491583891.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  allFoldSeq.append((np.array(sequences)[train_index], np.array(sequences)[test_index]))


In [54]:
np.save(basePath+'raw/allFoldSeq.npy',allFoldSeq)

/home/hejunliang/.conda/envs/hjlTorch1/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# 常见的一种存储方式，还未分割/padding到统一长度

In [55]:
def sequences2tl(sequences, trgpath):
    with open(trgpath, 'a', encoding='utf8') as f:
        for seq in tqdm.tqdm(sequences, 'write into file: '):
            questions, answers = seq
            seq_len = len(questions)
            f.write(str(seq_len) + '\n')
            f.write(','.join([str(q) for q in questions]) + '\n')
            f.write(','.join([str(a) for a in answers]) + '\n')

# 下面是我自己的存储方式，请根据自己要求存

In [56]:
def splitToMaxStep(sequences,maxstep):
    e_data=[]
    a_data=[]
    for e_features,a in tqdm.tqdm(sequences, 'splitting into MaxStep: '):
        
        length=e_features.shape[0]
        slices = length//maxstep + (1 if length%maxstep > 0 else 0)
        for i in range(slices):
            e_temp = np.zeros(shape=[maxstep,1])
            a_temp = np.zeros(shape=[maxstep,1])
            if length>0:
                if length>=maxstep:
                    l=maxstep
                else:
                    l=length
                for j in range(l):
                    e_temp[j]=e_features[i*maxstep+j]
                    a_temp[j]=a[i*maxstep+j]
                length = length - maxstep
            e_data.append(e_temp)
            a_data.append(a_temp)
    
    return np.concatenate((np.array(e_data).astype(float),np.array(a_data).astype(float)),axis=2)

In [57]:
allFoldSeq=np.load(basePath+'raw/allFoldSeq.npy',allow_pickle=True)
MAX_STEP = 128
i=1
for train_sequences,test_sequences in allFoldSeq:
    train_data=splitToMaxStep(train_sequences,MAX_STEP)
    test_data=splitToMaxStep(test_sequences,MAX_STEP)
    trainFile=open(basePath+'raw/train/train_data'+'_'+str(i)+'.txt','wb')
    testFile=open(basePath+'raw/test/test_data'+'_'+str(i)+'.txt','wb')
    pickle.dump(train_data,trainFile)
    pickle.dump(test_data,testFile)
    trainFile.close()
    testFile.close()
    i=i+1


splitting into MaxStep:   0%|          | 0/3330 [00:00<?, ?it/s]

splitting into MaxStep: 100%|██████████| 832/832 [00:00<00:00, 18053.83it/s]
